In [1]:
from itertools import combinations
from math import factorial

import pandas as pd
from shapely import intersection
from geopandas import GeoDataFrame, sjoin
from colocationpatterns.sample_data import generate_sample_data

In [2]:
class ColocationMiner:

    def __init__(self, data: GeoDataFrame, feature_type_column: str, feature_type_unique_id_column: str, neighbourhood: float):

        self.data = data # all events instances (id, event tpy, location, other attributes)
        self.feature_type_column = feature_type_column # column to recognize event type
        self.feature_type_unique_id_column = feature_type_unique_id_column # column with id unique within event type
        self.ET = set(data[feature_type_column].unique()) # set of event types
        self.R = neighbourhood # max distance to consider neighbour as colocation
        self.K = data[feature_type_column].nunique() # number of event types
        # self.table_ids_iterator = iter(range(0, sum(((factorial(self.K))//(factorial(i)*factorial(self.K-i)) for i in range(1, self.K+1)))))
        self.tables = {k: {} for k in range(1, self.K+1)} # structure to store tables based on size-k colocation

    def calculate_participation_ratio(self):

        pass

    def calculate_participation_index(self):

        pass

    def merge_by_neighbourhood(self, tables_id:tuple):

        result = cm.tables[1][tables_id[0]].copy()
        result['geometry'] = result['geometry'].buffer(self.R)
        
        for table_id in tables_id[1::]:
        
            table = cm.tables[1][table_id].copy()
            merged = sjoin(result, table)
            if merged.empty: return None
            merged.drop(columns='index_right', inplace=True)
            merged = pd.merge(merged, table, left_on=table_id, right_on=table_id)
            merged['geometry_y'] = merged['geometry_y'].buffer(self.R)
            merged['geometry'] = merged.apply(lambda row: intersection(row['geometry_x'], row['geometry_y']), axis=1)
            result = merged.drop(columns=['geometry_x', 'geometry_y'])
    
        return result
    
    
    def __call__(self):

        # Generate co-location candidates and compute statistics
        for k in range(1, self.K+1):
            
            for table_comb_ids in combinations(self.ET, k):

                if k == 1: # elementary tables
                    print('Creating elementary tables')
                    self.tables[k] =  { table_id:
                        GeoDataFrame(
                            [{
                                table_id: getattr(event, self.feature_type_unique_id_column),
                                'geometry': event.geometry
                            } for event in table[[self.feature_type_unique_id_column, 'geometry']].itertuples()]
                        ) for table_id, table in self.data.groupby(by=self.feature_type_column, as_index=False)
                    }
                    break

                

                print(f'Creating table k_level = {k} for tables {table_comb_ids}')
                table = self.merge_by_neighbourhood(table_comb_ids)
                
                self.tables[k][table_comb_ids] = table
            

            
       
            

In [3]:
data = generate_sample_data()

In [4]:
cm = ColocationMiner(data, 'spatial_feature_type', 'instance_id', 3)

In [5]:
cm()

Creating elementary tables
Creating table k_level = 2 for tables ('B', 'A')
Creating table k_level = 2 for tables ('B', 'C')
Creating table k_level = 2 for tables ('A', 'C')
Creating table k_level = 3 for tables ('B', 'A', 'C')


In [6]:
cm.tables

{1: {'A':    A           geometry
  0  1  POINT (1.75 1.75)
  1  2  POINT (5.45 3.75)
  2  3   POINT (8.5 3.25)
  3  4     POINT (4.25 5),
  'B':    B            geometry
  0  1     POINT (2.1 0.5)
  1  2  POINT (10.65 0.85)
  2  3   POINT (13.74 3.9)
  3  4   POINT (6.25 1.35)
  4  5      POINT (15.1 6),
  'C':    C          geometry
  0  1    POINT (8.65 1)
  1  2   POINT (0.1 3.3)
  2  3  POINT (14.2 5.5)},
 2: {('B',
   'A'):    B  A                                           geometry
  0  1  1  POLYGON ((-0.9 0.5, -0.88555 0.79405, -0.84236...
  1  4  3  POLYGON ((9.23555 1.05595, 9.19236 0.76473, 9....
  2  4  2  POLYGON ((3.30764 1.93527, 3.37918 2.22085, 3....,
  ('B',
   'C'):    B  C                                           geometry
  0  2  1  POLYGON ((9.23581 -1.79576, 8.98329 -1.64441, ...
  1  3  3  POLYGON ((11.24559 5.56671, 11.42097 5.80318, ...
  2  4  1  POLYGON ((9.23555 1.05595, 9.19236 0.76473, 9....
  3  5  3  POLYGON ((15.97085 3.12918, 15.68527 3.05764, ...,
  